In [ ]:
import sys

In [ ]:
sys.path.append('./src')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
cc_df = pd.read_csv('new_fraud.csv')

In [ ]:
def drop_func(data):
    data.drop(['merchant','first','last','street','city','state','zip','job','trans_num','unix_time'], axis = 1, inplace= True)
    return data

In [ ]:
cc_df = drop_func(cc_df)

In [ ]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long
0,2019-01-05 11:12,4302480582202070,grocery_pos,91.73,M,43.2326,-86.2492,128715,1995-05-25,42.533807,-86.610405
1,2019-12-23 19:06,3583635130604940,food_dining,48.03,F,40.0369,-75.0664,1526206,1985-01-01,39.594014,-74.548268
2,2019-08-17 21:26,374930071163758,shopping_net,8.45,M,42.2203,-83.3583,31515,1971-11-05,43.055047,-83.141898
3,2019-08-09 16:56,30273037698427,grocery_pos,50.56,M,35.5762,-91.4539,111,2000-06-13,36.495068,-92.304638
4,2019-05-20 13:35,4904681492230010,shopping_pos,22.69,F,41.8467,-74.1038,722,1990-10-19,41.058714,-74.070742


In [ ]:
def amt_z_func(data):
    amt_info = pd.read_pickle('./src/amt_info.pkl')
    data = data.merge(amt_info, on = 'cc_num', how = 'left')
    data['amt_z'] = (data['amt'] - data['mean']) / data['std']
    data.drop(['mean','std'], axis = 1, inplace = True)

    cat_info = pd.read_pickle('./src/cat_info.pkl')
    data = data.merge(cat_info, on = ['cc_num','category'], how = 'left')
    data['cat_amt_z'] = (data['amt'] - data['mean']) / data['std']
    data.drop(['mean','std'], axis =1 , inplace = True)

    return data

In [ ]:
cc_df = amt_z_func(cc_df)

In [ ]:
cc_df.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,amt_z,cat_amt_z
0,2019-01-05 11:12,4302480582202070,grocery_pos,91.73,M,43.2326,-86.2492,128715,1995-05-25,42.533807,-86.610405,NaN,NaN
1,2019-12-23 19:06,3583635130604940,food_dining,48.03,F,40.0369,-75.0664,1526206,1985-01-01,39.594014,-74.548268,NaN,NaN
2,2019-08-17 21:26,374930071163758,shopping_net,8.45,M,42.2203,-83.3583,31515,1971-11-05,43.055047,-83.141898,-0.319722,-0.253636
3,2019-08-09 16:56,30273037698427,grocery_pos,50.56,M,35.5762,-91.4539,111,2000-06-13,36.495068,-92.304638,-0.276392,-0.330353
4,2019-05-20 13:35,4904681492230010,shopping_pos,22.69,F,41.8467,-74.1038,722,1990-10-19,41.058714,-74.070742,NaN,NaN


In [ ]:
def hour_func(data):
    data['hour'] = pd.to_datetime(data['trans_date_trans_time']).dt.hour

    def hour_func(x):
        if (x >= 6) & (x < 12):
            return 'morning'
        elif (x >= 12) & (x < 18):
            return 'afternoon'
        elif (x >= 18) & (x < 23):
            return 'night'
        else:
            return 'evening'

    data['hour_cat'] = data['hour'].apply(hour_func)
    hour_cnt = pd.read_pickle('./src/hour_cnt.pkl')

    data = data.merge(hour_cnt, on = ['cc_num','hour_cat'], how = 'left')
    data.drop(['trans_date_trans_time', 'hour', 'hour_cat'], axis =1 , inplace = True)

    return data

In [ ]:
cc_df = hour_func(cc_df)

In [ ]:
cc_df.head()

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,amt_z,cat_amt_z,hour_perc
0,4302480582202070,grocery_pos,91.73,M,43.2326,-86.2492,128715,1995-05-25,42.533807,-86.610405,NaN,NaN,NaN
1,3583635130604940,food_dining,48.03,F,40.0369,-75.0664,1526206,1985-01-01,39.594014,-74.548268,NaN,NaN,NaN
2,374930071163758,shopping_net,8.45,M,42.2203,-83.3583,31515,1971-11-05,43.055047,-83.141898,-0.319722,-0.253636,0.284069
3,30273037698427,grocery_pos,50.56,M,35.5762,-91.4539,111,2000-06-13,36.495068,-92.304638,-0.276392,-0.330353,0.406857
4,4904681492230010,shopping_pos,22.69,F,41.8467,-74.1038,722,1990-10-19,41.058714,-74.070742,NaN,NaN,NaN


In [ ]:
from geopy.distance import distance

def dist_func(data):
    data['distance'] = data.apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis = 1)
    dist_info = pd.read_pickle('./src/dist_info.pkl')

    data = data.merge(dist_info, on = 'cc_num', how = 'left')
    data['dist_z'] = (data['distance'] - data['mean']) / data['std']
    data.drop(['lat','long','merch_lat','merch_long','mean','std'], axis = 1, inplace = True)

    return data

In [ ]:
cc_df = dist_func(cc_df)

In [ ]:
cc_df.head()

,cc_num,category,amt,gender,city_pop,dob,amt_z,cat_amt_z,hour_perc,distance,dist_z
0,4302480582202070,grocery_pos,91.73,M,128715,1995-05-25,NaN,NaN,NaN,83.048320,NaN
1,3583635130604940,food_dining,48.03,F,1526206,1985-01-01,NaN,NaN,NaN,66.228673,NaN
2,374930071163758,shopping_net,8.45,M,31515,1971-11-05,-0.319722,-0.253636,0.284069,94.411700,0.686319
3,30273037698427,grocery_pos,50.56,M,111,2000-06-13,-0.276392,-0.330353,0.406857,127.567201,1.682793
4,4904681492230010,shopping_pos,22.69,F,722,1990-10-19,NaN,NaN,NaN,87.559408,NaN


In [ ]:
def age_func(data):
    data['dob'] = pd.to_datetime(data['dob']).dt.year
    return data

In [ ]:
cc_df = age_func(cc_df)

In [ ]:
cc_df.head()

,cc_num,category,amt,gender,city_pop,dob,amt_z,cat_amt_z,hour_perc,distance,dist_z
0,4302480582202070,grocery_pos,91.73,M,128715,1995,NaN,NaN,NaN,83.048320,NaN
1,3583635130604940,food_dining,48.03,F,1526206,1985,NaN,NaN,NaN,66.228673,NaN
2,374930071163758,shopping_net,8.45,M,31515,1971,-0.319722,-0.253636,0.284069,94.411700,0.686319
3,30273037698427,grocery_pos,50.56,M,111,2000,-0.276392,-0.330353,0.406857,127.567201,1.682793
4,4904681492230010,shopping_pos,22.69,F,722,1990,NaN,NaN,NaN,87.559408,NaN


In [ ]:
def one_func(data):
    data = pd.get_dummies(data, drop_first = True)
    data.drop('cc_num', axis = 1, inplace = True)

    return data

In [ ]:
cc_df = one_func(cc_df)

In [ ]:
cc_df.head()

,amt,city_pop,dob,amt_z,cat_amt_z,hour_perc,distance,dist_z,category_food_dining,category_gas_transport,...,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
0,91.73,128715,1995,NaN,NaN,NaN,83.048320,NaN,0,0,...,0,0,0,0,0,0,0,0,0,1
1,48.03,1526206,1985,NaN,NaN,NaN,66.228673,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
2,8.45,31515,1971,-0.319722,-0.253636,0.284069,94.411700,0.686319,0,0,...,0,0,0,0,0,0,1,0,0,1
3,50.56,111,2000,-0.276392,-0.330353,0.406857,127.567201,1.682793,0,0,...,0,0,0,0,0,0,0,0,0,1
4,22.69,722,1990,NaN,NaN,NaN,87.559408,NaN,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
import sys
import pandas as pd
import numpy as np

In [ ]:
sys.path.append('./src')

In [ ]:
cc_df = pd.read_csv('new_fraud.csv')

In [ ]:
"['merchant' 'first' 'last' 'street' 'city' 'state' 'zip' 'job' 'trans_num'\n 'unix_time']

In [ ]:
cc_df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long
0,2019-01-05 11:12,4302480582202070,fraud_Padberg-Welch,grocery_pos,91.73,David,Rodriguez,M,821 Solis Points,Muskegon,...,49440,43.2326,-86.2492,128715,Historic buildings inspector/conservation officer,1995-05-25,000014ca3f6921fe6793f88fe494f39d,1325761935,42.533807,-86.610405
1,2019-12-23 19:06,3583635130604940,fraud_Zboncak Ltd,food_dining,48.03,Crystal,Gamble,F,899 Michele View Suite 960,Philadelphia,...,19149,40.0369,-75.0664,1526206,Structural engineer,1985-01-01,00001ded488fddab97677128e5034d39,1356289600,39.594014,-74.548268
2,2019-08-17 21:26,374930071163758,"fraud_Reichert, Rowe and Mraz",shopping_net,8.45,Daniel,Escobar,M,61390 Hayes Port,Romulus,...,48174,42.2203,-83.3583,31515,Police officer,1971-11-05,0000246d803d5f465cc322d8a3c3528f,1345238776,43.055047,-83.141898
3,2019-08-09 16:56,30273037698427,"fraud_Lockman, West and Runte",grocery_pos,50.56,Andrew,Patterson,M,06959 Stephen Branch Suite 246,Thida,...,72165,35.5762,-91.4539,111,Careers information officer,2000-06-13,0000258ae973a6199fca79d94947672f,1344531393,36.495068,-92.304638
4,2019-05-20 13:35,4904681492230010,fraud_Jast-McDermott,shopping_pos,22.69,Lisa,Lowe,F,574 David Locks Suite 207,Cottekill,...,12419,41.8467,-74.1038,722,Comptroller,1990-10-19,0000425d184356a21be4b39933c2c0ea,1337520926,41.058714,-74.070742


In [ ]:
from drop_func import drop_func
from amt_z_func import amt_z_func
from hour_func import hour_func
from dist_func import dist_func
from age_func import age_func
from one_func import one_func

def pre_func(data):
    data = drop_func(data)
    data = amt_z_func(data)
    data = hour_func(data)
    data = dist_func(data)
    data = age_func(data)
    data = one_func(data)

In [ ]:
cc_df.head()

,amt,city_pop,dob,amt_z,cat_amt_z,hour_perc,distance,dist_z,category_food_dining,category_gas_transport,...,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
0,91.73,128715,1995,NaN,NaN,NaN,83.048320,NaN,0,0,...,0,0,0,0,0,0,0,0,0,1
1,48.03,1526206,1985,NaN,NaN,NaN,66.228673,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
2,8.45,31515,1971,-0.319722,-0.253636,0.284069,94.411700,0.686319,0,0,...,0,0,0,0,0,0,1,0,0,1
3,50.56,111,2000,-0.276392,-0.330353,0.406857,127.567201,1.682793,0,0,...,0,0,0,0,0,0,0,0,0,1
4,22.69,722,1990,NaN,NaN,NaN,87.559408,NaN,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
script = """
from drop_func import drop_func
from amt_z_func import amt_z_func
from hour_func import hour_func
from dist_func import dist_func
from age_func import age_func
from one_func import one_func

def pre_func(data):
    data = drop_func(data)
    data = amt_z_func(data)
    data = hour_func(data)
    data = dist_func(data)
    data = age_func(data)
    data = one_func(data)

    return data
"""

In [ ]:
filename = './src/pre_func.py'

with open(filename, "w") as file:
    file.write(script)

In [ ]:
import sys
import pandas as pd

In [ ]:
cc_df = pd.read_csv('new_fraud.csv')

In [ ]:
sys.path.append('./src')

In [ ]:
from pre_func import pre_func

In [ ]:
cc_df = pre_func(cc_df)

In [ ]:
cc_df.head()

,amt,city_pop,dob,amt_z,cat_amt_z,hour_perc,distance,dist_z,category_food_dining,category_gas_transport,...,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
0,91.73,128715,1995,NaN,NaN,NaN,83.048320,NaN,0,0,...,0,0,0,0,0,0,0,0,0,1
1,48.03,1526206,1985,NaN,NaN,NaN,66.228673,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
2,8.45,31515,1971,-0.319722,-0.253636,0.284069,94.411700,0.686319,0,0,...,0,0,0,0,0,0,1,0,0,1
3,50.56,111,2000,-0.276392,-0.330353,0.406857,127.567201,1.682793,0,0,...,0,0,0,0,0,0,0,0,0,1
4,22.69,722,1990,NaN,NaN,NaN,87.559408,NaN,0,0,...,0,0,0,0,0,0,0,1,0,0
